# Python機械学習クックブック

In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets

## 特徴量の次元削減(特徴量抽出)
特徴量をまとめて、次元数を落とす

### 主成分を用いた特徴量削減

In [3]:
# データをロード
digits = datasets.load_digits()

# 特徴量行列を標準化
features = StandardScaler().fit_transform(digits.data)

# 99%の分散を維持するPCAクラスを作成
pca = PCA(n_components=0.99, whiten=True)

# PCAを実行
features_pca = pca.fit_transform(features)

# 結果を表示
print("元の特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_pca.shape[1])

元の特徴量数: 64
削減後の特徴量数: 54


### データが線形分離不可能な際の特徴量削減

In [4]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles

# 線形分離不可能なデータを作成
features, _ = make_circles(n_samples=1000, random_state=1, noise=0.1, factor=0.1)

# RBFカーネルを使用したKernelPCAで非線形次元削減を実行
kpca = KernelPCA(kernel="rbf", gamma=15, n_components=1)
features_kpca = kpca.fit_transform(features)

# 結果を表示
print("元の特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_kpca.shape[1])

元の特徴量数: 2
削減後の特徴量数: 1


### クラスの分離性最大化による特徴量削減

In [7]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

iris = datasets.load_iris()
features = iris.data
target = iris.target

# LDAオブジェクトを作成し、LDAを実行して特徴量を変換
lda = LinearDiscriminantAnalysis(n_components=1)
features_lda = lda.fit(features, target).transform(features)

# 結果を表示
print("元の特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_lda.shape[1])

# 変数の重要度を表す
lda.explained_variance_ratio_

元の特徴量数: 4
削減後の特徴量数: 1


array([0.9912126])

In [8]:
# 必要な主成分の数を決める
# n_componentsをNoneに設定すると、全ての主成分の寄与率が分かる
lda = LinearDiscriminantAnalysis(n_components=None)
features_lda = lda.fit(features, target)

# 各主成分の寄与率を格納
lda_var_ratios = lda.explained_variance_ratio_

# 関数を定義
def select_n_components(var_ratio, goal_var: float) -> int:
    # 分散の合計
    total_variance = 0.0

    # 主成分の数
    n_components = 0

    # 分散の合計が目標の分散の合計に達するまでループ
    for explained_variance in var_ratio:

        # 分散を分散の合計に加算
        total_variance += explained_variance

        # 主成分の数をカウント
        n_components += 1

        # 目標の分散の合計に達したらループを終了
        if total_variance >= goal_var:
            break

    # 主成分の数を返す
    return n_components

# 関数を実行
select_n_components(lda_var_ratios, 0.95)


1

### 行列因子分解による特徴量削減

In [9]:
from sklearn.decomposition import NMF

digits = datasets.load_digits()
features = digits.data

# NMFを作成し、特徴量を変換
nmf = NMF(n_components=10, random_state=1)
features_nmf = nmf.fit_transform(features)

# 結果を表示
print("元の特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_nmf.shape[1])

元の特徴量数: 64
削減後の特徴量数: 10


/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1692: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


### 疎データの特徴量削減

In [10]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

digits = datasets.load_digits()

# 特徴量行列の標準化
features = StandardScaler().fit_transform(digits.data)

# 疎行列を作成
features_sparse = csr_matrix(features)

# TSVDを作成し、特徴量を変換
tsvd = TruncatedSVD(n_components=10)

# 疎行列をTSVDに適合
features_sparse_tsvd = tsvd.fit(features_sparse).transform(features_sparse)

# 結果を表示
print("元の特徴量数:", features_sparse.shape[1])
print("削減後の特徴量数:", features_sparse_tsvd.shape[1])


元の特徴量数: 64
削減後の特徴量数: 10


In [11]:
# 必要な主成分の数を決める
# 特徴量数-1を指定するとしてTSVDを実行し、元データへの寄与率を所望の値に達するために必要な成分の数を計算する
tsvd = TruncatedSVD(n_components=features_sparse.shape[1]-1)
features_tsvd = tsvd.fit(features)

# 各主成分の寄与率を格納
tsvd_var_ratios = tsvd.explained_variance_ratio_

# 関数を定義
def select_n_components(var_ratio, goal_var: float) -> int:
    # 説明された寄与率の累計変数を初期化
    total_variance = 0.0

    # 主成分の数を初期化
    n_components = 0

    # 各主成分の寄与率をループ
    for explained_variance in var_ratio:
            
            # 分散を分散の合計に加算
            total_variance += explained_variance
    
            # 主成分の数をカウント
            n_components += 1
    
            # 目標の分散の合計に達したらループを終了
            if total_variance >= goal_var:
                break

    # 主成分の数を返す
    return n_components

# 関数を実行
select_n_components(tsvd_var_ratios, 0.95)

40

## 特徴量の次元削減(特徴量選択)
有効な特徴量をピックアップし、意味のない特徴量を捨てることで次元数を落とす

### 数値特徴量の分散による閾値処理
分散の小さい(あまり情報を持たない)特徴量を捨てる

In [13]:
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris()
features = iris.data
target = iris.target

# 分散が0.5より大きい特徴量を削除
thresholder = VarianceThreshold(threshold=.5)

# 分散を計算
features_high_variance = thresholder.fit_transform(features)

# 分散の大きい特徴量を表示
features_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

### 2値特徴量の分散閾値処理

In [15]:
from sklearn.feature_selection import VarianceThreshold

features = [[0, 1, 0],
            [0, 1, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0]]

thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
thresholder.fit_transform(features)

array([[0],
       [1],
       [0],
       [1],
       [0]])

### 強く相関した特徴量の取り扱い

In [21]:
features = np.array(
    [
        [1, 1, 1],
        [2, 2, 0],
        [3, 3, 1],
        [4, 4, 0],
        [5, 5, 1],
        [6, 6, 0],
        [7, 7, 1],
        [8, 7, 0],
        [9, 7, 1],
    ]
)

# 特徴量行列をDataFrameに変換
dataframe = pd.DataFrame(features)

# 相関行列を作成
corr_matrix = dataframe.corr().abs()

# 相関行列の上三角（右上）行列を取得
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# 相関係数が0.95より大きい特徴量のインデックスを取得
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# 特徴量を削除
dataframe.drop(dataframe.columns[to_drop], axis=1)

<ipython-input-21-d013847759a4>:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


,0,2
0,1,1
1,2,0
2,3,1
3,4,0
4,5,1
5,6,0
6,7,1
7,8,0
8,9,1


### クラス分類に無関係な特徴量の削除

In [22]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

iris = datasets.load_iris()
features = iris.data
target = iris.target

# データを整数に変換して、カテゴリ特徴量を作成
features = features.astype(int)

# カイ二乗統計量を計算
chi2_selector = SelectKBest(chi2, k=2)

# カイ二乗統計量を計算し、特徴量を2つ選択
features_kbest = chi2_selector.fit_transform(features, target)

# 結果を表示
print("元の特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_kbest.shape[1])



元の特徴量数: 4
削減後の特徴量数: 2


In [24]:
# F値を計算
fvalue_selector = SelectKBest(f_classif, k=2)

# 最も高いF値をもつ特徴量を2つ選択
features_kbest = fvalue_selector.fit_transform(features, target)

# 結果を表示
print("元の特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_kbest.shape[1])

元の特徴量数: 4
削減後の特徴量数: 2


In [27]:
from sklearn.feature_selection import SelectPercentile

# 特徴量の個数ではなく、上位n%という形で指定して特徴量を選択する事もできる
# F値の上位75%の特徴量を選択
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

# 結果を表示
print("元の特徴量数:", features.shape[1])
print("削減後の特徴量数:", features_kbest.shape[1])

元の特徴量数: 4
削減後の特徴量数: 3


### 再帰的な特徴量の除去

In [29]:
import warnings
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# 警告を無視
warnings.filterwarnings(
    action="ignore", module="scipy", message="^internal gelsd"
)

# 特徴量行列とターゲットベクトルを作成
features, target = make_regression(
    n_samples=10000, n_features=100, n_informative=2, random_state=1
)

# 線形回帰器を作成
ols = linear_model.LinearRegression()

# 再帰的に特徴量を削減
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

# 最適な特徴量の個数
rfecv.n_features_

# 最適な特徴量を表示
rfecv.support_

# 最適な特徴量のランキングを表示
rfecv.ranking_

array([84, 81, 85, 75, 77,  1, 98, 36, 57, 10, 60, 69, 52,  8, 14, 65, 23,
        6, 82, 93, 48, 50, 30, 72, 25, 68, 63,  7, 67, 80, 29, 31, 78, 95,
       49, 88, 21, 91,  9,  1, 42, 20, 89, 19, 40, 90, 92, 12, 58, 47, 71,
       66, 28, 70, 38, 32,  4,  5, 62, 11, 56, 94, 59, 26, 74, 97, 18, 37,
       41, 55, 79, 53, 13, 86, 34,  2, 61, 46, 45, 51, 83, 54,  3, 27, 15,
       22, 64, 99, 96, 43, 24, 33, 17, 39, 87, 73, 35, 44, 16, 76])